## Flask Web Framework


[Flask](https://flask.palletsprojects.com/en/1.1.x/) is a [micro web framework](https://en.wikipedia.org/wiki/Flask_%28web_framework%29) written in Python. Function decorators are used in Flask to achieve _routes to functions_ mapping. We first show how a simple service works, and then show how to load a model (e.g., based on pytorch) and serve it as well.

### Example: Weather Reporting Service

The key thing to see here are that the HTTP route `/` is mapped directly to a function `weather`. For instance, when someone hits `localhost:5000` (5000 is the default unless specified in `app.run()` below) the function `weather` starts execution based on any received inputs.

This service can be run by using the command `python weather.py` (assuming that is the filename for the above script) locally. If the port 5000 is open, then this server will be accessible to the world if the server has an external IP address.


In [ ]:
# load Flask 
import flask
from flask import jsonify
from geopy.geocoders import Nominatim
import requests

app = flask.Flask(__name__)

# define a predict function as an endpoint 
@app.route("/", methods=["GET","POST"])
def weather():

    data = {"success": False}
    #https://pypi.org/project/geopy/
    geolocator = Nominatim(user_agent="cloud_function_weather_app")


    params = flask.request.json
    if params is None:
        params = flask.request.args


    # params = request.get_json()
    if "msg" in params:
        location = geolocator.geocode(str(params['msg']))
        data['location'] = [location.address,location.latitude,location.longitude,location.altitude]
        # https://www.weather.gov/documentation/services-web-api
        try:
            result1 = requests.get(f"https://api.weather.gov/points/{location.latitude},{location.longitude}")
            result2 = requests.get(f"{result1.json()['properties']['forecast']}")
            data["response"] = result2.json()
            data["success"] = True
        except:
            pass
    return jsonify(data)
    
# start the flask app, allow remote connections
if __name__ == '__main__':
    app.run(host='0.0.0.0')

### Example: Food Image Service

In this example, our app uses the requests library to make a query to an external service (that just produces random images of food) and sends back that information to the client.

In [ ]:
 # load Flask 
import flask
import requests

app = flask.Flask(__name__)

@app.route("/", methods=["GET"])
def food():

    data = {"success": False}

    if "msg" in flask.request.args:
        data['foodname'] = str(flask.request.args['msg'])
        try:
            req = requests.get(f"https://foodish-api.herokuapp.com/api/images/{data['foodname']}")
            data["response"] = req.json()
            data["success"] = True
        except:
            pass
    else:
        try:
            req = requests.get("https://foodish-api.herokuapp.com/api/")
            data["response"] = req.json()
            data["success"] = True
        except:
            pass

    if data['success']:
        img_str= f"""
                      <img src="{data["response"]['image']}" alt="Random Food" width="500" height="600"> 
                 """
    return f"""
                <!doctype html>
                <html>
                <head>
                <title>Our Funky HTML Page</title>
                <meta name="description" content="Our first page">
                <meta name="keywords" content="html tutorial template">
                <link rel="stylesheet" href="https://unpkg.com/flexgrid.io@3.0.4/dist/flexgrid.min.css" />

                </head>
                <body>
                 <div class="row xs-justify-center xs-items-center">

                    {img_str}

                 </div>
                </body>
                </html>
        """
    
# start the flask app, allow remote connections
if __name__ == '__main__':
    app.run(host='0.0.0.0')

### Model Serving

We can use a similar idea to serve the recommendation model we built earlier as follows.

In [ ]:
 # load Flask 
import flask
from recommend_pytorch_base import MF #comment this out if running in this ipynb. else create a recommend_pytorch_base.py file based on the training cell above.
from recommend_pytorch_inf import get_top_n, get_previously_seen #comment this out if running in this ipynb. else create a recommend_pytorch_inf.py file based on the training cell above.
import torch
import pandas as pd
import surprise
import time



app = flask.Flask(__name__)

start_time = time.time()

#data preload
data                = surprise.Dataset.load_builtin('ml-1m')
trainset            = data.build_full_trainset()
testset             = trainset.build_anti_testset()
movies_df           = pd.read_csv('./movies.dat',sep="::",header=None,engine='python')
movies_df.columns   = ['iid','name','genre']
movies_df.set_index('iid',inplace=True)

#model preload
k           = 10 #latent dimension
c_bias      = 1e-6
c_vector    = 1e-6
model       = MF(trainset.n_users, trainset.n_items, k=k, c_bias=c_bias, c_vector=c_vector)
model.load_state_dict(torch.load('./recommendation_model_pytorch.pkl'))
model.eval()

print('Model and data preloading completed in ', time.time()-start_time)

@app.route("/", methods=["GET"])
def recommend():

    data = {"success": False}

    if "uid" in flask.request.args:

        data['uid'] = str(flask.request.args['uid'])

        try:
            data['seen'] = get_previously_seen(trainset, data['uid'], movies_df)
            recommended = get_top_n(model, testset, trainset, data['uid'], movies_df, n=10)
            print(recommended)
            data['recommended'] = [x[1] for x in recommended]
            data["success"] = True
        except:
            pass

    return flask.jsonify(data)
    
# start the flask app, allow remote connections
if __name__ == '__main__':
    app.run(host='0.0.0.0')

We can use the following request in the browser `http://0.0.0.0:5000/?uid=196` or use the requests module. 